# Microsoft Interpret ML によるモデル開発とモデル解釈
製造プロセスから採取されたセンサーデータと検査結果のデータを用いて、品質管理モデルを構築します。

## 1. データ準備

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv("data/Factory.csv")

In [2]:
from sklearn.model_selection import train_test_split

X = df.drop(columns=["Quality","ID"],axis=1)
y = df["Quality"].values

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.1,random_state=100,stratify=y)

## 2. データ探索

In [3]:
from interpret import show
from interpret.data import ClassHistogram

hist = ClassHistogram().explain_data(X_train, y_train, name = 'Train Data')
show(hist)

<!-- http://127.0.0.1:7054/4700787320/ -->

## 3. Explainable Boosting Machine (EBM) によるモデル学習

In [4]:
from interpret.glassbox import ExplainableBoostingClassifier, LogisticRegression, ClassificationTree, DecisionListClassifier
seed = 1234
ebm = ExplainableBoostingClassifier(random_state=seed)
ebm.fit(X_train, y_train)   #Works on dataframes and numpy arrays

WARNING - Passing a numpy array to schema autogen when it should be dataframe.


ExplainableBoostingClassifier(binning_strategy='uniform',
               data_n_episodes=2000, early_stopping_run_length=50,
               early_stopping_tolerance=1e-05,
               feature_names=['ProcessA-Pressure', 'ProcessA-Humidity', 'ProcessA-Vibration', 'ProcessB-Light', 'ProcessB-Skill', 'ProcessB-Temp', 'ProcessB-Rotation', 'ProcessC-Density', 'ProcessC-PH', 'ProcessC-skewness', 'ProcessC-Time'],
               feature_step_n_inner_bags=0,
               feature_types=['continuous', 'continuous', 'continuous', 'continuous', 'continuous', 'continuous', 'continuous', 'continuous', 'continuous', 'continuous', 'continuous'],
               holdout_size=0.15, holdout_split=0.15, interactions=0,
               learning_rate=0.01, max_tree_splits=2,
               min_cases_for_splits=2, n_estimators=16, n_jobs=-2,
               random_state=1234, schema=None, scoring=None,
               training_step_episodes=1)

## 4. モデル解釈 (グローバル)

In [5]:
ebm_global = ebm.explain_global(name='EBM')
show(ebm_global)

<!-- http://127.0.0.1:7054/4734427432/ -->

## 5. モデル解釈 (ローカル)
テストデータから算出された予測値の解釈を行います。

In [12]:
ebm_local = ebm.explain_local(X_test[:20], y_test[:20], name='EBM')
show(ebm_local)

<!-- http://127.0.0.1:7054/4743089792/ -->

##  6. EBM モデルの精度確認

In [7]:
from interpret.perf import ROC

ebm_perf = ROC(ebm.predict_proba).explain_perf(X_test, y_test, name='EBM')
show(ebm_perf)

<!-- http://127.0.0.1:7054/4743470608/ -->

## 7. ロジスティック回帰、決定木モデルの学習

In [8]:
from interpret.glassbox import LogisticRegression, ClassificationTree

# We have to transform categorical variables to use Logistic Regression and Decision Tree
X_enc = pd.get_dummies(X, prefix_sep='.')
feature_names = list(X_enc.columns)
X_train_enc, X_test_enc, y_train, y_test = train_test_split(X_enc, y, test_size=0.20, random_state=seed)

lr = LogisticRegression(random_state=seed, feature_names=feature_names, penalty='l1')
lr.fit(X_train_enc, y_train)

tree = ClassificationTree()
tree.fit(X_train_enc, y_train)

/Users/konabuta/miniconda3/envs/explain/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning:

Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.



## 8. ダッシュボードでのモデル比較

In [9]:
lr_perf = ROC(lr.predict_proba).explain_perf(X_test_enc, y_test, name='Logistic Regression')
tree_perf = ROC(tree.predict_proba).explain_perf(X_test_enc, y_test, name='Classification Tree')

show(lr_perf)
show(tree_perf)
show(ebm_perf)

<!-- http://127.0.0.1:7054/4743088504/ -->

<!-- http://127.0.0.1:7054/4734336192/ -->

<!-- http://127.0.0.1:7054/4743470608/ -->

## 9. モデル解釈の比較 (グローバル、ローカル)

### Global Explain

In [10]:
lr_global = lr.explain_global(name='LR')
tree_global = tree.explain_global(name='Tree')

show(lr_global)
show(tree_global)
show(ebm_global)

<!-- http://127.0.0.1:7054/4734397240/ -->

<!-- http://127.0.0.1:7054/4742511976/ -->

<!-- http://127.0.0.1:7054/4734427432/ -->

### Local Explainn

In [17]:
lr_local = lr.explain_local(X_test[:20], y_test[:20], name='LR')
tree_local = tree.explain_local(X_test[:20], y_test[:20], name='Tree')

show(lr_local)
show(tree_local)
show(ebm_local)

<!-- http://127.0.0.1:7054/4817150976/ -->

<!-- http://127.0.0.1:7054/4831330824/ -->

<!-- http://127.0.0.1:7054/4743089792/ -->

### ダッシュボード

In [20]:
# Do everything in one shot with the InterpretML Dashboard by passing a list into show

show([hist, lr_global, lr_local, lr_perf, tree_global, tree_local, tree_perf, ebm_global, ebm_local, ebm_perf], share_tables=True)

<!-- http://127.0.0.1:7054/4831152072/ -->
 Open in new window

WARNING - Pane index 1 larger than selected explanations.
WARNING - Pane index 1 larger than selected explanations.
